In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.stat import Correlation
#import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
#import holidays
from datetime import datetime, timezone
from pyspark.ml.classification import RandomForestClassifier, BinaryLogisticRegressionSummary
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.pipeline import PipelineModel

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("US_Accidents") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "2g") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR") #supress warnings

/opt/conda/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/11/25 14:02:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/25 14:02:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/25 14:02:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/11/25 14:02:09 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
# Read in Data
df = spark.read.parquet("updated_dataset.parquet")

In [4]:
# Use StringIndexer for encoding the 'Severity' column
indexer = StringIndexer(inputCol="Severity", outputCol="SeverityIndex")
df = indexer.fit(df).transform(df)

In [5]:
# Create list of features
feature_list = []
for col in df.columns:
    if col == 'Severity':
        continue
    elif col == 'SeverityIndex':
        continue
    elif col == 'Severity_Binary':
        continue
    else:
        feature_list.append(col)

In [6]:
# Split the data into train and test
splits = df.randomSplit([0.8, 0.2], 314)
train = splits[0]
test = splits[1]

In [7]:
# Undersampling
from pyspark.sql import functions as F

# Step 1: Group by 'Severity' and count occurrences
class_counts = train.groupBy("SeverityIndex").count()

# Step 2: Use PySpark's min() function to find the minimum count
min_class_size = class_counts.agg(F.min('count')).collect()[0][0]

undersampled_train_list = []

for row in class_counts.collect():
    class_label = row['SeverityIndex']
    class_size = row['count']

    if class_size > min_class_size:
        # Sample the data for this class to the size of the minimum class
        class_data = train.filter(F.col("SeverityIndex") == class_label)
        class_data_undersampled = class_data.sample(withReplacement=False, fraction=min_class_size / class_size)
    else:
        # For classes that are already at the minimum size, keep all samples
        class_data_undersampled = train.filter(F.col("SeverityIndex") == class_label)

    undersampled_train_list.append(class_data_undersampled)

# Combine all the undersampled DataFrames
undersampled_train = undersampled_train_list[0]  # start with the first one
for df in undersampled_train_list[1:]:
    undersampled_train = undersampled_train.union(df)

# Show the result
undersampled_train.show()

# Step 4: Group by 'Severity' and count the occurrences in the undersampled DataFrame
undersampled_class_counts = undersampled_train.groupBy("SeverityIndex").count()

# Show the result
undersampled_class_counts.show()

+--------+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+-------------+
|Severity|Temperature|Humidity|Pressure|Visibility|Wind_Speed|Precipitation|Weekday|Rush_Hour|Rain|Snow|    SeasonVec|Astronomical_TwilightIndex|Interstate_Indicator|Sex_ratio|Percent_Age_65_over|MedianIncome|Urban_Ratio|Traffic_Interference|Traffic_Intersection|Destination|Percent_Age_15-24|Severity_Binary|SeverityIndex|
+--------+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+-------------+
|       2|        -14|      

+-------------+-----+
|SeverityIndex|count|
+-------------+-----+
|          0.0|52196|
|          1.0|52209|
|          3.0|52151|
|          2.0|52117|
+-------------+-----+



In [8]:
# Assemble data for logistic regression model
assembler = VectorAssembler(inputCols=feature_list,
                            outputCol="features")

undersampled_train = assembler.transform(undersampled_train)
test = assembler.transform(test)

In [9]:
# Standardize the predictors
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(undersampled_train)
scaledTrainData = scalerModel.transform(undersampled_train)
scaledTestData = scalerModel.transform(test)

In [10]:
# Fit logistic regression model with intercept
from pyspark.ml.classification import LogisticRegression

# instantiate the model
lr = LogisticRegression(labelCol='SeverityIndex',
                        featuresCol='scaledFeatures',
                        #maxIter=10, 
                        #regParam=0.3, 
                        #elasticNetParam=0.8,
                        family="multinomial")

# Fit the model
lrModel = lr.fit(scaledTrainData)
print(f"Detected number of classes: {lrModel.numClasses}")

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

Detected number of classes: 4
Coefficients: DenseMatrix([[-0.01334403, -0.07128954,  0.02587489, -0.03296679, -0.01341369,
               0.00418653, -0.03827505, -0.0414086 , -0.0052609 ,  0.03859128,
               0.53378849,  0.3361789 ,  0.02685283, -0.00533664, -0.10402565,
               0.02338114,  0.02509545,  0.00830775,  0.04131541, -0.00853421,
              -0.0281566 ,  0.05434903, -0.00207331],
             [-0.15187889, -0.08080775,  0.24197059, -0.00124228,  0.13794727,
               0.00918346, -0.06362724, -0.02058515,  0.0427827 ,  0.03308882,
               0.2165488 ,  0.15220874, -0.12827583, -0.12458854,  0.46909713,
               0.04690592, -0.13731518,  0.00295971,  0.29095483, -0.00111609,
              -0.25522731, -0.09860735, -0.03574083],
             [-0.23879821, -0.04744912,  0.0156794 ,  0.0303126 ,  0.06258713,
              -0.00967514, -0.13688315, -0.16821018,  0.0124914 ,  0.04114373,
               0.21979831,  0.14243572, -0.07372698,  0.16

In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# compute predictions. this will append column "prediction" to dataframe
lrPred = lrModel.transform(scaledTestData)
lrPred.select("prediction").distinct().show()

evaluator = MulticlassClassificationEvaluator(labelCol='SeverityIndex', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(lrPred, {evaluator.metricName: "accuracy"})
print(f'Accuracy: {accuracy}')

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
|       3.0|
|       2.0|
+----------+



Accuracy: 0.38801369765569205


In [12]:
precision = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedPrecision'})
recall = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedRecall'})
f1_score = evaluator.evaluate(lrPred, {evaluator.metricName: 'f1'})

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')

Precision: 0.7678097147434838
Recall: 0.38801369765569205
F1 Score: 0.4783962037309053


In [13]:
# Add a column to indicate correct or incorrect predictions
predictions = lrPred.withColumn(
    'is_correct', F.expr("CASE WHEN SeverityIndex = prediction THEN 1 ELSE 0 END")
)

# Calculate accuracy by class
accuracy_by_class = predictions.groupBy('SeverityIndex').agg(
    (F.sum('is_correct') / F.count('SeverityIndex')).alias('accuracy')
)

# Show per-class accuracy
accuracy_by_class.show()

+-------------+-------------------+
|SeverityIndex|           accuracy|
+-------------+-------------------+
|          0.0|  0.359959066337807|
|          1.0|  0.500798622819316|
|          3.0| 0.7773843430066969|
|          2.0|0.42339902852995365|
+-------------+-------------------+



In [14]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import functions as F

# Compute predictions
lrPred = lrModel.transform(scaledTestData)

# Show distinct predictions
lrPred.select("prediction").distinct().show()

# Evaluate overall accuracy
evaluator = MulticlassClassificationEvaluator(labelCol='SeverityIndex', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(lrPred)
print(f'Accuracy: {accuracy}')

# Calculate metrics by class
labels = lrPred.select('SeverityIndex').distinct().orderBy('SeverityIndex').rdd.flatMap(lambda x: x).collect()

metrics = {}
for label in labels:
    # Filter predictions for the current label
    true_positive = lrPred.filter((F.col('SeverityIndex') == label) & (F.col('prediction') == label)).count()
    false_positive = lrPred.filter((F.col('SeverityIndex') != label) & (F.col('prediction') == label)).count()
    false_negative = lrPred.filter((F.col('SeverityIndex') == label) & (F.col('prediction') != label)).count()
    true_negative = lrPred.filter((F.col('SeverityIndex') != label) & (F.col('prediction') != label)).count()

    # Precision, Recall, and F1 Score
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    # Store metrics
    metrics[label] = {'precision': precision, 'recall': recall, 'f1_score': f1_score}

# Print metrics for each class
for label, metric in metrics.items():
    print(f"Class {label} - Precision: {metric['precision']:.4f}, Recall: {metric['recall']:.4f}, F1 Score: {metric['f1_score']:.4f}")

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
|       3.0|
|       2.0|
+----------+



Accuracy: 0.38801369765569205


Class 0.0 - Precision: 0.8874, Recall: 0.3600, F1 Score: 0.5122
Class 1.0 - Precision: 0.3219, Recall: 0.5008, F1 Score: 0.3919
Class 2.0 - Precision: 0.0626, Recall: 0.4234, F1 Score: 0.1091
Class 3.0 - Precision: 0.0285, Recall: 0.7774, F1 Score: 0.0551
